# Method Resolution Order (MRO) in Python

**Role:** Senior Python Engineer
**Context:** Object-Oriented Programming (Inheritance & Polymorphism)

## Overview

**Method Resolution Order (MRO)** is the defined rule set that Python follows to determine which method to call when a method is invoked on an object.

Because Python supports **Multiple Inheritance** (a class inheriting from more than one parent), it needs a strict algorithm to decide where to look for a method first. If it didn't have this, it wouldn't know which parent class to prioritize if two parents had the same method name.

### How to View MRO

You can inspect the MRO of any class programmatically using the `.mro()` method or the `__mro__` attribute.

```python
print(ChildClass.mro())
# Output: [<class '__main__.Child'>, <class '__main__.Parent'>, <class 'object'>]

```

---

## 1. The Hierarchy of `object`

In Python 3, **every** class implicitly inherits from the built-in `object` base class. If you don't specify a parent, Python assigns `object` automatically. This is why `object` always appears at the very end of an MRO list.

---

## 2. Inheritance Scenarios & MRO

### A. Single Inheritance

**Structure:** Child inherits from Parent.
**Rule:** Look in Child → Parent → Object.

```python
class Parent:
    def show(self): print("Parent")

class Child(Parent):
    pass

# MRO: Child -> Parent -> object

```

### B. Multi-Level Inheritance

**Structure:** GrandChild inherits from Child, which inherits from Parent.
**Rule:** Climb the ladder directly up.

**Order:** `GrandChild` → `Child` → `Parent` → `object`.

### C. Multiple Inheritance

**Structure:** Child inherits from two distinct parents: `class Child(A, B)`.
**Rule:** Left-to-Right. Python looks at the first parent listed in the parentheses, then the second.

```python
class A:
    def show(self): print("A")

class B:
    def show(self): print("B")

class Child(A, B):
    pass

# MRO: Child -> A -> B -> object
# Because we defined it as Child(A, B), A comes first.

```

---

## 3. The "Diamond Problem" (Hybrid Inheritance)

This is the complex scenario mentioned in the lecture where multiple paths lead back to a common ancestor.

**Structure:**

* `B` inherits from `A`.
* `Y` inherits from `X`.
* Both `A` and `X` inherit from a common top parent `P`.
* `C` inherits from both `B` and `Y`.

### The C3 Linearization Algorithm

Python uses the **C3 Linearization** algorithm to solve this. The golden rule of C3 is:
**"Children before Parents, Left before Right."**

Specifically, it ensures that a parent class (`P`) is never checked until **all** of its children (`A` and `X`) have been checked.

**The Flow for Class C:**

1. **Check C** (The instance itself).
2. **Check B** (Left parent).
3. **Check A** (Parent of B).
4. **Check Y** (Right parent).
5. **Check X** (Parent of Y).
6. **Check P** (The common grandparent).
7. **Check object**.

**Why doesn't it go A -> P immediately?**
Because `P` is also a parent of `X`. If Python visited `P` before `X`, it would violate the rule that "children (`X`) must be checked before parents (`P`)." Therefore, it pauses the left branch at `A` and switches to the right branch to finish `Y` and `X` before finally visiting `P`.

### Summary of Rules

| Inheritance Type | MRO Rule |
| --- | --- |
| **Single** | Child → Parent |
| **Multiple** | Left Parent → Right Parent |
| **Diamond/Hybrid** | Child → Left Branch (Children only) → Right Branch (Children only) → Common Parent |